In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
import gc
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier, LGBMRegressor
import lightgbm as lgb
import optuna
import numpy as np

In [2]:
df_train = pd.read_csv("D:\\COMPI-TOP\\Song-popularity\\train.csv")
df_test = pd.read_csv("D:\\COMPI-TOP\\Song-popularity\\test.csv")
submission = pd.read_csv("D:\\COMPI-TOP\\Song-popularity\\sample_submission.csv")

In [3]:
train = df_train.copy()
test = df_test.copy()

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                40000 non-null  int64  
 1   song_duration_ms  35899 non-null  float64
 2   acousticness      36008 non-null  float64
 3   danceability      35974 non-null  float64
 4   energy            36025 non-null  float64
 5   instrumentalness  36015 non-null  float64
 6   key               35935 non-null  float64
 7   liveness          35914 non-null  float64
 8   loudness          36043 non-null  float64
 9   audio_mode        40000 non-null  int64  
 10  speechiness       40000 non-null  float64
 11  tempo             40000 non-null  float64
 12  time_signature    40000 non-null  int64  
 13  audio_valence     40000 non-null  float64
 14  song_popularity   40000 non-null  int64  
dtypes: float64(11), int64(4)
memory usage: 4.6 MB


In [5]:
train.head()

,id,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence,song_popularity
0,0,212990.0,0.642286,0.856520,0.707073,0.002001,10.0,NaN,-5.619088,0,0.082570,158.386236,4,0.734642,0
1,1,NaN,0.054866,0.733289,0.835545,0.000996,8.0,0.436428,-5.236965,1,0.127358,102.752988,3,0.711531,1
2,2,193213.0,NaN,0.188387,0.783524,-0.002694,5.0,0.170499,-4.951759,0,0.052282,178.685791,3,0.425536,0
3,3,249893.0,0.488660,0.585234,0.552685,0.000608,0.0,0.094805,-7.893694,0,0.035618,128.715630,3,0.453597,0
4,4,165969.0,0.493017,NaN,0.740982,0.002033,10.0,0.094891,-2.684095,0,0.050746,121.928157,4,0.741311,0


In [11]:
useful_features = [c for c in train.columns if c not in ("id", "song_popularity")]
object_cols = [col for col in train[useful_features].columns if train[useful_features].dtypes[col]=='object']
print(object_cols)

[]


In [12]:
pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('scale', RobustScaler())
])

In [13]:
train_ = pd.DataFrame(columns=train[useful_features].columns, data=pipeline.fit_transform(train[useful_features]))
test_ = pd.DataFrame(columns=test[useful_features].columns, data=pipeline.fit_transform(test[useful_features]))

In [14]:
train_.insert(0, 'id', train.id)
test_.insert(0, 'id', test.id)

In [15]:
train = train_.copy()
test = test_.copy()
del train_
gc.collect()
del test_
gc.collect()

0

In [16]:
cat_cols = ['key', 'time_signature', 'audio_mode']
cat_indices = []
for f in cat_cols:
    if f in train.columns:
        idx = list(train.columns).index(f)
        cat_indices.append(idx)

In [17]:
train['song_popularity'] = df_train['song_popularity']
train.head()

,id,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence,song_popularity
0,0,0.625453,1.310314,0.952114,0.008803,0.013607,0.833333,0.000000,0.168801,0.0,0.332190,1.414573,1.0,0.376253,0
1,1,0.000000,-0.223716,0.479554,0.440540,-0.494693,0.500000,3.486191,0.257609,1.0,0.889668,-0.350325,0.0,0.312228,1
2,2,0.155662,0.000000,-1.610004,0.265720,-2.359470,0.000000,0.400888,0.323892,0.0,-0.044798,2.058552,0.0,-0.480076,0
3,3,1.502061,0.909126,-0.088200,-0.510026,-0.690593,-0.833333,-0.477306,-0.359828,0.0,-0.252212,0.473309,0.0,-0.402338,0
4,4,-0.491502,0.920504,0.000000,0.122755,0.029653,0.833333,-0.476309,0.850908,0.0,-0.063910,0.257984,1.0,0.394729,0


In [18]:
target = ['song_popularity']
not_features = ['id','kfold', 'song_popularity']
cols = list(train.columns)
features = [feat for feat in cols if feat not in not_features]

In [19]:
train["kfold"] = -1
train_targets = train[target]

In [20]:
# initialize stratified k-fold
kf = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
for fold, (train_indicies, valid_indicies) in enumerate(kf.split(X=train, y=train_targets)):
    train.loc[valid_indicies, "kfold"] = fold

# *Optuna for Lightgbm*

In [100]:
def run(trial):
    fold = 0
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)

    ytrain = xtrain.song_popularity
    yvalid = xvalid.song_popularity
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    
    param = {
        "task": "train",
        "objective": "regression",
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "device": "gpu",
        "gpu_platform_id": 0,
        "gpu_device_id": 0,
        "learning_rate" : trial.suggest_float("learning_rate", 1e-2, 0.25, log=True),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    model = LGBMRegressor(**param, random_state=42, nestimators=10000)
    model.fit(xtrain, ytrain, eval_set = [(xvalid, yvalid)],verbose=False,
            categorical_feature=cat_indices,
            callbacks=[ 
                       lgb.early_stopping(stopping_rounds=2000)
                      ])
    preds_valid = model.predict(xvalid)
    roc = roc_auc_score(yvalid, preds_valid)
    return roc

In [101]:
study = optuna.create_study(direction="maximize")
study.optimize(run, n_trials=150)

[I 2022-01-20 19:57:51,172] A new study created in memory with name: no-name-ba650e84-48a9-4bf1-9395-b66d7b1c1f2c
C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [6, 9, 12]
  _log_warning('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] Unknown parameter: nestimators
[LightGBM] [Warning] feature_fraction is set=0.677052383832905, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.677052383832905
[LightGBM] [Warning] bagging_fraction is set=0.6564400817026375, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6564400817026375
[LightGBM] [Warning] lambda_l1 is set=8.128139137200383e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.128139137200383e-05
[LightGBM] [Warning] lambda_l2 is set=3.6004590669690253e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.6004590669690253e-08
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 2000 rounds


[I 2022-01-20 19:58:00,093] Trial 0 finished with value: 0.5700431642327128 and parameters: {'learning_rate': 0.05436138984881357, 'lambda_l1': 8.128139137200383e-05, 'lambda_l2': 3.6004590669690253e-08, 'num_leaves': 231, 'feature_fraction': 0.677052383832905, 'bagging_fraction': 0.6564400817026375, 'bagging_freq': 5, 'min_child_samples': 86}. Best is trial 0 with value: 0.5700431642327128.


...

Did not meet early stopping. Best iteration is:
[60]	valid_0's auc: 0.579638
[LightGBM] [Warning] Unknown parameter: nestimators
[LightGBM] [Warning] feature_fraction is set=0.6519993082650116, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6519993082650116
[LightGBM] [Warning] bagging_fraction is set=0.49053981190416573, subsample=1.0 will be ignored. Current value: bagging_fraction=0.49053981190416573
[LightGBM] [Warning] lambda_l1 is set=2.536644167119355e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.536644167119355e-08
[LightGBM] [Warning] lambda_l2 is set=0.00012376518601228217, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.00012376518601228217
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [6, 9, 12]
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 2000 rounds


[I 2022-01-20 20:02:37,961] Trial 149 finished with value: 0.5805258679065981 and parameters: {'learning_rate': 0.012882591291006256, 'lambda_l1': 2.536644167119355e-08, 'lambda_l2': 0.00012376518601228217, 'num_leaves': 82, 'feature_fraction': 0.6519993082650116, 'bagging_fraction': 0.49053981190416573, 'bagging_freq': 5, 'min_child_samples': 85}. Best is trial 73 with value: 0.5889527849202367.


Did not meet early stopping. Best iteration is:
[67]	valid_0's auc: 0.580526


In [102]:
study.best_params

{'learning_rate': 0.018508133231121448,
 'lambda_l1': 5.842694522644729e-08,
 'lambda_l2': 0.02217395184292706,
 'num_leaves': 76,
 'feature_fraction': 0.5745960798262617,
 'bagging_fraction': 0.5288996205050897,
 'bagging_freq': 5,
 'min_child_samples': 77}

In [31]:
scores = []
final_test_predictions = []
for fold in range(10):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)

    ytrain = xtrain.song_popularity
    yvalid = xvalid.song_popularity
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    
    params_lgb = {  
                "task": "train",
                "objective": "regression",
                "metric": "auc",
                "verbosity": -1,
                "boosting_type": "gbdt",
                "device": "gpu",
                "gpu_platform_id": 0,
                "gpu_device_id": 0,
                'learning_rate': 0.018508133231121448,
                'lambda_l1': 5.842694522644729e-08,
                'lambda_l2': 0.02217395184292706,
                'num_leaves': 76,
                'feature_fraction': 0.5745960798262617,
                'bagging_fraction': 0.5288996205050897,
                'bagging_freq': 5,
                'min_child_samples': 77,
                'n_estimators': 10000
             }
    lgb_train = lgb.Dataset(xtrain, ytrain)
    lgb_val = lgb.Dataset(xvalid, yvalid)
    
    model = lgb.train(params=params_lgb,
                      train_set=lgb_train,
                      valid_sets=lgb_val,
                      early_stopping_rounds=3000,
                      verbose_eval=False)
   
    preds_valid = model.predict(xvalid,num_iteration=model.best_iteration)
    test_predict = model.predict(test[features],num_iteration=model.best_iteration)
    #preds_valid = model.predict(xvalid, num_iteration=model.best_iteration)
    #test_preds = model.predict(test[features], num_iteration=model.best_iteration)
    final_test_predictions.append(test_predict)
    roc = roc_auc_score(yvalid, preds_valid)
    print("ROC-AUC is: ", roc)
    print(fold, roc)
    
    scores.append(roc)

print(np.mean(scores))

C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


ROC-AUC is:  0.5803466579364782
0 0.5803466579364782


C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


ROC-AUC is:  0.5755706042749675
1 0.5755706042749675


C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


...

ROC-AUC is:  0.5777853865754906
7 0.5777853865754906


C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


ROC-AUC is:  0.5685139856177533
8 0.5685139856177533


C:\Users\Sarkhel\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


ROC-AUC is:  0.5685676789065779
9 0.5685676789065779
0.5757139462300663


# *Submission*

In [32]:
submission.song_popularity = np.mean(np.column_stack(final_test_predictions), axis=1)

In [34]:
submission.head()

,id,song_popularity
0,0,0.417862
1,1,0.436469
2,2,0.318221
3,3,0.338337
4,4,0.382079


In [35]:
submission.to_csv("D:\\COMPI-TOP\\Song-popularity\\Sub_LGBMReg1.csv", index=False)